In [1]:
import torch
from torch import Tensor
from torch import nn
from torch import optim
from torch.nn  import functional as F 
import numpy as np

import torchvision
from torchvision import transforms
from IPython import display

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

## Import the Adjoint Method (ODE Solver)
from torchdiffeq import odeint_adjoint as odeint

def get_accuracy(dataloader, model, device='cpu'):
    tot = len(dataloader.dataset)
    with torch.no_grad():
        n_right = 0
        for data in dataloader:
            batch = data[0].to(device)
            labels = data[1].to(device)
            
            outputs = model(batch)
            out_labels = torch.argmax(outputs, dim=1)
            n_right += torch.sum(labels == out_labels)
    return float(n_right) / tot

Device: cpu


In [45]:
class ODE_Function(nn.Module):
    def __init__(self, i_size, h_size, o_size):
        super(ODE_Function, self).__init__()
        self.f1 = nn.Linear(i_size, h_size)
        self.f2 = nn.ReLU(inplace=True)        
        self.f3 = nn.Linear(h_size, o_size)
        self.nfe = 0
        
    def forward(self, t, x):
        self.nfe +=1
        out = self.f1(x)
        out = self.f2(out)
        out = self.f3(out)
        return out 
    
class NOde(nn.Module):
    def __init__(self, odefunc):
        super(NOde, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()
        
    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(self.odefunc, x, self.integration_time, rtol=1e-7, atol=1e-9)
        return out
        
    

In [56]:
X = np.array([[0, 0], [0, 1],
                [1, 0], [1, 1]])

# Set the labels, the correct results for the xor operation
y = np.array([0, 1, 
                 1, 0])

X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float).unsqueeze(1)

net = NOde(ODE_Function(2, 2, 1))

lr = 0.1
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=lr)

prediction, w = net(X)
print(prediction)
print(w)
loss = criterion(prediction, y)

optimizer.zero_grad()
loss.backward()
optimizer.step()

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], grad_fn=<SelectBackward>)
tensor([[0.7900, 0.7900],
        [1.0973, 2.0973],
        [2.0748, 1.0748],
        [2.3820, 2.3820]], grad_fn=<SelectBackward>)


ValueError: Target and input must have the same number of elements. target nelement (4) != input nelement (8)

In [63]:
class Neural_Ode(nn.Module):
    
    def __init__(self, i_size, h_size, o_size):
        super(Neural_Ode, self).__init__()
        self.i_size = i_size
        self.h_size = h_size
        self.o_size = o_size
        self.f1 = nn.Linear(i_size, h_size)
        self.f2 = nn.ReLU(inplace=True)        
        self.f3 = nn.Linear(h_size, o_size)
        self.integration_time = torch.tensor([0, 1]).float()
        
    def func(i_size, h_size, o_size):
        out = self.f1(x)
        out = self.f2(out)
        out = self.f3(out)
        return out 
        
    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(func, x, self.integration_time, rtol=1e-7, atol=1e-9)
        return out
    
    
    

In [64]:
X = np.array([[0, 0], [0, 1],
                [1, 0], [1, 1]])

# Set the labels, the correct results for the xor operation
y = np.array([0, 1, 
                 1, 0])

X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float).unsqueeze(1)

net1 = Neural_Ode(2, 2, 1)

lr = 0.1
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=lr)

prediction, w = net(X)
print(prediction)
print(w)
loss = criterion(prediction, y)

optimizer.zero_grad()
loss.backward()
optimizer.step()

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], grad_fn=<SelectBackward>)
tensor([[0.7900, 0.7900],
        [1.0973, 2.0973],
        [2.0748, 1.0748],
        [2.3820, 2.3820]], grad_fn=<SelectBackward>)


ValueError: Target and input must have the same number of elements. target nelement (4) != input nelement (8)